<div>
<img style="float:left; border-radius:50%" src="https://avatars2.githubusercontent.com/u/58553327?s=460&u=3276252f07fb379c248bc8c9ce344bfdcaed7c45&v=4" width="40px">
<a href="https://github.com/carlosholivan"><img src="https://www.sferalabs.cc/wp-content/uploads/github-logo.png" width=70px style="float: right;"></a>
</div>

<a name="top"></a>

# ANALYSIS OF SPECTRAL CENTROIDS OF LONDON PHILARMONIC ORCHESTRA DATASET

Author: Carlos Hernández Oliván<br>
Last update: 7 November 2020

<div class="alert alert-block alert-warning">
<strong>Note:</strong> Audio format files: wav (in different resolutions), flac.
<br>
</div>   

## 1. SPECTRAL CENTROID

The spectral centroid is a measure that indicates where the center of mass of the spectrum is located.

It is calculated as the weighted mean of the frequencies present in the signal, determined using a Fourier transform, with their magnitudes as the weights:

\begin{equation}
\mathrm{COG} = \frac{\sum_{x=a}^b\!\mu_{A}(x)x}{\sum_{x=a}^b\!\mu_{A}(x)}
%\label{eq:cogCont}
\end{equation}

In [1]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
# wav and MIDI paths
path = '/media/carlos/FILES/INVESTIGACION/Datasets/London Philarmonic Orchestra/'

Convert mp3 files in wav files and save them to disk

In [3]:
### Checking format of audio files: if extension is mp3, try to convert it to wav
from pydub import AudioSegment

def convert_mp3_to_wav(path):
    instruments = os.listdir(path)
    for i in range(len(instruments)):
        instrument_path = os.path.join(path, instruments[i])
        instrument_files = os.listdir(instrument_path)
        for j in range(len(instrument_files)):
            instrument_files_path = os.path.join(instrument_path, instrument_files[j])
            if instrument_files_path.split(".",1)[1] == 'mp3':
                try:
                    sound = AudioSegment.from_mp3(instrument_files_path)
                    instrument_files_wav_path = instrument_files_path.split(".",1)[0]
                    sound.export(instrument_files_wav_path + ".wav", format="wav") #convert to wav file
                    if instrument_files_path.split(".",1)[1] == 'mp3':
                        os.remove(instrument_files_path) #delete mp3 examples
                except:
                    continue

In [4]:
# Check if the selected instrument is in the dataset and get path if True
def get_instruments_path(path, instrument_name):
    instruments = os.listdir(path)
    if instrument_name in instruments:
        for filename in instruments:
            if filename == instrument_name:
                instrument_path = os.path.join(path, filename)
                instrument_files = os.listdir(instrument_path)
        return instrument_path, instrument_files
    else:
        raise ValueError('The instrument is not in the dataset')

In [5]:
def compute_centroids(path, instrument_name):
    # Store all instrument file names and their centroids in a dictionary
    
    instrument_path, instrument_files = get_instruments_path(path, instrument_name)
    
    data = {}
    for j in range(len(instrument_files)):
        instrument_file_path = os.path.join(instrument_path, instrument_files[j])
        try:
            y, sr = librosa.load(str(instrument_file_path), sr=None)   
            centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
            instrument_name = instrument_files[j].split(".",1)[0]
            if instrument_name not in data:
                data.update({instrument_name: centroid})
        except:
            continue
    return data

In [6]:
def subplot_centroids(instrument_name):
    
    # Compute centroids
    data = compute_centroids(path, instrument_name)
    
    # Plot the centroids
    if len(data.keys()) == 1:
        times = librosa.times_like(list(data.values())[0])
        plt.plot(times, list(data.values())[0].T, label=list(data.keys())[0])
            
    else:
        if len(data.keys()) < 5:
            n_rows = 1
            n_cols = len(data.keys())
            
        else:
            n_rows = len(data.keys())//5 + len(data.keys())%5 - 1
            n_cols = 5

        fig, ax = plt.subplots(n_rows, n_cols, figsize=(20, len(data.keys())))
        fig.subplots_adjust(hspace = .5, wspace=.01)
        ax = ax.ravel()

        i = 0
        for label, centroid_value in data.items():
            if i == len(data.keys()):
                break
            times = librosa.times_like(centroid_value)
            ax[i].plot(times, centroid_value.T, label=label)
            ax[i].tick_params(axis="x", labelsize=8)
            ax[i].tick_params(axis="y", labelsize=8)
            ax[i].set_title(label, fontsize=8)
            i += 1

In [7]:
def plot_centroids_together(instrument_name):
    
    # Compute centroids
    data = compute_centroids(path, instrument_name)
    
    fig, ax = plt.subplots(figsize=(15,8))
    plt.xlabel('time', fontsize=15)
    plt.ylabel('freq', fontsize=15)
    for label, centroid_value in data.items():
        times = librosa.times_like(centroid_value)
        ax.plot(times, centroid_value.T, label=label)
        ax.legend(loc='upper right')
    ax.set(title='Centroids')

### Plot results

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output

d = widgets.Dropdown(options=os.listdir(path), value=None)
    
display(d)   

def on_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        clear_output()
        instrument_name = change['new']
        _, instr_path = get_instruments_path(path, instrument_name)
        display(d)
        print('Number of instruments of', instrument_name, 'family is:', len(instr_path))
        subplot_centroids(instrument_name)

instrument_name = d.observe(on_change)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '/media/carlos/FILES/INVESTIGACION/Datasets/London Philarmonic Orchestra/'

[<button type="button" class="btn btn-primary" style="background-color:#a273f9; border:0">⇦ Back to Top</button>](#top)

### References <a name="references"></a>

* Librosa

<div>
    
<button type="button" class="btn btn-primary" style="float:left; background-color:#a273f9; border:0"><a href="#top" style="color:white; text-decoration: none">⇦ Back to Top</a></button>

<button type="button" class="btn btn-primary" style="float:right; background-color:#BA55D3; border:0"><a href="2-chord_prediction.ipynb" style="color:white; text-decoration: none">Go to Chord Detection ⇒</a></button>
    
</div>